In [1]:
import os

In [2]:
%pwd

'a:\\mlflow\\Heart-Failure-Prediction-with-Machine-Learning-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'a:\\mlflow\\Heart-Failure-Prediction-with-Machine-Learning-MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir:str
    train_data_path:Path
    test_data_path:Path
    model_names:str
    model_param:str
    target_column:str

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
config = read_yaml(PARAMS_FILE_PATH)
print(config)

[2025-07-22 17:55:00,142 : INFO: yaml file: params.yaml loaded successfully]
{'Models': {'Support_Vector_Machine': {'gamma': 0.01, 'C': 15, 'kernel': 'rbf'}, 'Knearest_Neighbour': {'n_neighbours': 20}, 'AdaBoost': {'n_estimators': 100}}}


In [12]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.model_trainer.root_dir])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        param = self.params.Models
        schema = self.schema.TARGET_COLUMN
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_names=config.model_names,
            model_param=param,
            target_column=schema.name
        )

        return model_trainer_config

In [13]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble  import AdaBoostClassifier 
import pandas as pd
import os
import joblib
from mlProject import logger

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):

        train_data =  pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        y_train = train_data[self.config.target_column]

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        # SVM
        SVM = SVC(
            gamma=self.config.model_param.Support_Vector_Machine.gamma,
            C=self.config.model_param.Support_Vector_Machine.C,
            kernel=self.config.model_param.Support_Vector_Machine.kernel,
            random_state=42 
        )
        SVM.fit(X_train, y_train)
        joblib.dump(SVM, os.path.join(self.config.root_dir, self.config.model_names.model1))

        # KNN
        KNN=KNeighborsClassifier(n_neighbors=self.config.model_param.K_Nearest_Neighbours.n_neighbours)
        KNN.fit(X_train, y_train)
        joblib.dump(KNN, os.path.join(self.config.root_dir, self.config.model_names.model2))

        # AdaBoost
        AdaBoost= AdaBoostClassifier(n_estimators=self.config.model_param.AdaBoost.n_estimators,
                                   random_state=42
        )
        AdaBoost.fit(X_train, y_train)
        joblib.dump(AdaBoost, os.path.join(self.config.root_dir, self.config.model_names.model3))
        

In [15]:
try:
    configuration_manager = ConfigurationManager()
    model_trainer_config = configuration_manager.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-07-22 18:18:49,366 : INFO: yaml file: config\config.yaml loaded successfully]
[2025-07-22 18:18:49,370 : INFO: yaml file: params.yaml loaded successfully]
[2025-07-22 18:18:49,373 : INFO: yaml file: schema.yaml loaded successfully]
[2025-07-22 18:18:49,374 : INFO: created directory at: artifacts/model_trainer]
